# 3章 二値分類における評価指標

## 3.3 混同行列 (Confusion Matrix)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# 文字列や数値で表されたラベルを，0~(ラベル種類数-1)に変換して特徴を上書きする関数
def label_encoding(df):
    # object(今回の場合は文字列)のカラムを取得
    object_type_column_list = [c for c in df.columns if df[c].dtypes=='object']
    for object_type_column in object_type_column_list:
        # LabelEncoderを使ってカテゴリで示されるデータ(質的変数)を数値に変換
        label_encoder = LabelEncoder()
        df[object_type_column] = label_encoder.fit_transform(df[object_type_column])
    return df

# 特徴を標準化する関数
def standard_scale(X_train, X_val):
    ss = StandardScaler()
    # 学習用データの特徴を標準化する
    X_train = ss.fit_transform(X_train)
    # 検証用データの特徴を標準化する
    X_val = ss.transform(X_val)
    return X_train, X_val

# 前処理を実行した学習用データを返す関数
def run_preprocess(train_df):
    # Nanが含まれており、ロジスティック回帰がそのままでは動かないためNanを含むカラムを削除する
    # 本来はNanを埋めるなどの処理をした方が分類性能は上がることが多いが、
    # 本書では興味の対象ではないため、説明を簡単にするためにカラムごと削除する
    remove_column_list = ['previous_year_rating', 'education']
    train_df.drop(remove_column_list, axis=1, inplace=True)
    train_df = label_encoding(train_df)
    return train_df

# train.csvデータをpandasで読み込む
train_df = pd.read_csv('../data/train.csv')

# 前処理する
train_df = run_preprocess(train_df)
# 教師ラベルを取り出す
y = train_df.pop('is_promoted').values

# 学習用データと検証用データを分離する
X_train, X_val, y_train, y_val = train_test_split(train_df, y, test_size=0.33, random_state=42)
# 学習用データと検証用データを標準化する
X_train, X_val = standard_scale(X_train, X_val)

# ロジスティック回帰モデルを学習する
clf = LogisticRegression(max_iter=200, random_state=2020)
clf.fit(X_train, y_train)
# ロジスティック回帰モデルで推論を行う
y_val_hat = clf.predict(X_val)

# 混同行列を作成する
conf_matrix = confusion_matrix(y_val, y_val_hat, labels=[1, 0])
df_confusion_matrix = pd.DataFrame(conf_matrix, columns=["Positive", "Negative"], index=["Positive", "Negative"])

# 作成した混同行列をヒートマップの形で描画する
sns.heatmap(df_confusion_matrix, fmt="d", annot=True, cmap="Blues", annot_kws={"fontsize": 20})
plt.xlabel("model output class")
plt.ylabel("actual class")
plt.show()

## 3.4.1 Employee Promotion Data で正解率を算出

In [ ]:
from sklearn.metrics import accuracy_score

# 検証用データの正解率を求める
accuracy = accuracy_score(y_val, y_val_hat)
print(f"正解率: {round(accuracy, 2)}")

## 3.5.1 Employee Promotion Data でMCCを算出

In [ ]:
from sklearn.metrics import matthews_corrcoef

# 検証用データのマシューズ相関係数を算出する
mcc = matthews_corrcoef(y_val, y_val_hat)
print(f"MCC: {round(mcc, 2)}")

In [ ]:
import numpy as np

# 実際の正解クラスと予測されたクラスを逆にして検証用データのマシューズ相関係数を算出する
mcc = matthews_corrcoef(np.where(y_val == 0, 1, 0), np.where(y_val_hat == 0, 1, 0))
print(f"MCC: {round(mcc, 2)}")

## 3.6.1 Employee Promotion Data で適合率を算出

In [ ]:
from sklearn.metrics import precision_score

# 検証用データの適合率を算出する
precision = precision_score(y_val, y_val_hat)
print(f"適合率: {round(precision, 2)}")

## 3.7.1 Employee Promotion Data で再現率を算出

In [ ]:
from sklearn.metrics import recall_score

# 検証用データの再現率を算出する
recall = recall_score(y_val, y_val_hat)
print(f"再現率: {round(recall, 2)}")

## 3.8.1 Employee Promotion Data でF1スコアを算出

In [ ]:
from sklearn.metrics import f1_score

# 検証用データのF1スコアを算出する
f1 = f1_score(y_val, y_val_hat)
print(f"F1スコア: {round(f1, 2)}")

## 3.9.1 Employee Promotion Data でG-Meanを算出

In [ ]:
import math

from sklearn.metrics import confusion_matrix

# G-Meanを算出するための関数
def g_mean_score(y, y_hat):
    # 混同行列を作成して各要素を変数に格納する
    tn, fp, fn, tp = confusion_matrix(y, y_hat).ravel()
    # True Positive Rateを算出する
    tp_rate =  tp / (tp + fn)
    print("True Positive Rate:", round(tp_rate, 2))
    # True Negative Rateを算出する
    tn_rate = tn / (tn + fp)
    print("True Negative Rate:", round(tn_rate, 2))
    # G-meanはTrue Positive RateとTrue Negative Rateの幾何平均を計算
    return math.sqrt(tp_rate * tn_rate)

print("G-Mean: ", round(g_mean_score(y_val, y_val_hat), 2))

## 3.10.1 Employee Promotion Data でROC-AUCを算出

In [ ]:
from sklearn.metrics import roc_auc_score

# ラベルが1である予測確率を取得する
y_val_hat = clf.predict_proba(X_val)[:, 1]

# ROC-AUCを算出する
roc_auc = roc_auc_score(y_val, y_val_hat)
print(f"ROC-AUC: {round(roc_auc, 2)}")

## 3.11.2 Employee Promotion Data でPR-AUCを算出

In [ ]:
from sklearn.metrics import auc, precision_recall_curve

# ラベルが1である予測確率を取得する
y_val_hat = clf.predict_proba(X_val)[:, 1]

# 予測確率を使って閾値を操作したPrecisionとRecallとその時の閾値の3つを算出
precision, recall, thresholds = precision_recall_curve(y_val, y_val_hat)
# AUCを算出する
pr_auc = auc(recall, precision)

print(f"PR-AUC: {round(pr_auc, 2)}")

In [ ]:
import random

# 実行した時に同じ値になるように乱数のシードを固定する
random.seed(2022)
# 0から１の間の小数点つきの数値をy_valの長さ分だけ生成する
random_predict = [random.random() for _ in range(len(y_val))]

# ランダムに生成した値を使って閾値を操作したPrecisionとRecallとその時の閾値の3つを算出
precision, recall, thresholds = precision_recall_curve(y_val, random_predict)
# AUCを算出する
pr_auc = auc(recall, precision)

print(f"PR-AUC(Random Model): {round(pr_auc, 2)}")

## 3.12 pAUC (partial AUC)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

# ラベルが1である予測確率を取得する
y_val_hat = clf.predict_proba(X_val)[:, 1]
# 閾値を操作したときのFalse Positive Rate と True Positive Rateとその時の閾値の値を算出
fpr, tpr, thresholds = roc_curve(y_val, y_val_hat)
# ROC曲線を描画しつつ、AUCの値も凡例に記載する
plt.plot(fpr, tpr, label="ROC curve (area = %0.2f)" % roc_auc_score(y_val, y_val_hat),)

# 0~0.2の範囲でのpAUCの計算
p_auc = roc_auc_score(y_val, y_val_hat, max_fpr=0.2)
# False Positive Rate が0.2以下の範囲でのFalse Positive Rate と True Positive Rateのみ取得する
pfpr = fpr[fpr <= 0.2]
ptpr = tpr[fpr <= 0.2]
# pAUCの対象範囲の線を描画する。
plt.plot(pfpr, ptpr, label="pAUC curve (area = %0.2f)" % p_auc,)
# pAUCの対象範囲を塗りつぶす
plt.fill_between(pfpr, ptpr, 0, facecolor='darkorange', alpha=0.5)
# 凡例を表示する
plt.legend(loc="lower right")
# 横軸の名前をFalse positive rateにする
plt.xlabel("False positive rate")
# 縦軸の名前をTrue positive rateにする
plt.ylabel("True positive rate")
plt.title("ROC Curve")
# pAUCの塗りつぶし部分に"pAUC"というテキストを表示する
plt.text(0.05, 0.15, "pAUC")
plt.show()

## 3.12.1 Employee Promotion Data でpAUCを算出

In [ ]:
from sklearn.metrics import roc_auc_score

# ラベルが1である予測確率を取得する
y_val_hat = clf.predict_proba(X_val)[:, 1]

# pAUCを算出
p_auc = roc_auc_score(y_val, y_val_hat, max_fpr=0.2)
print(f"pAUC: {round(p_auc, 2)}")

## 3.13 クラスの分布の変化による評価指標への影響

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import auc, precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
from imblearn.under_sampling import RandomUnderSampler

import pandas as pd


# 検証用データのクラスの分布の比率を操作するための関数(weightsは1つ目が Positive の比率, 2つ目が Negative の比率を表す)
def calc_custom_storategy(y_val, weights=(1, 1)):
    # NegativeとPositiveの数を算出する
    is_promoted_count = np.bincount(y_val)
    strategy = {0: weights[0] * is_promoted_count[1], 1: weights[1] * is_promoted_count[1]}
    return strategy

# ここまでで紹介した全ての評価指標で評価を行う関数
def evaluate(X_resampled, y_resampled):
    y_val_hat = clf.predict(X_resampled)
    accuracy = accuracy_score(y_resampled, y_val_hat)
    print(f"accuracy: {round(accuracy, 2)}")

    precision = precision_score(y_resampled, y_val_hat)
    print(f"precision: {round(precision, 2)}")

    recall = recall_score(y_resampled, y_val_hat)
    print(f"recall: {round(recall, 2)}")

    f1score = f1_score(y_resampled, y_val_hat)
    print(f"f1 score: {round(f1score, 2)}")

    mcc = matthews_corrcoef(y_resampled, y_val_hat)
    print(f"MCC: {round(mcc, 2)}")

    g_mean = g_mean_score(y_resampled, y_val_hat)
    print(f"G-Mean: {round(g_mean, 2)}")

    y_val_hat =clf.predict_proba(X_resampled)[:, 1]
    roc_auc = roc_auc_score(y_resampled, y_val_hat)
    print(f"ROC-AUC: {round(roc_auc, 2)}")

    precision, recall, thresholds = precision_recall_curve(y_resampled, y_val_hat)
    pr_auc = auc(recall, precision)
    print(f"PR-AUC: {round(pr_auc, 2)}")

    p_auc = roc_auc_score(y_resampled, y_val_hat, max_fpr=0.2)
    print(f"pAUC: {round(p_auc, 2)}")

# 学習データをデータフレームで読み込む
train_df = pd.read_csv('../data/train.csv')

# 前処理を行う
train_df = run_preprocess(train_df)
# 教師ラベルを取り出す
y = train_df.pop('is_promoted').values

# 学習用データと検証用データを分離する
X_train, X_val, y_train, y_val = train_test_split(train_df, y, test_size=0.33, random_state=42)
X_train, X_val = standard_scale(X_train, X_val)

# ロジスティック回帰モデルを学習する
clf = LogisticRegression(max_iter=200, random_state=2020)
clf.fit(X_train, y_train)

# 評価用のデータのクラスの分布をPositive: Negative = 1:1にする
strategy = calc_custom_storategy(y_val, weights=(1, 1))
rus = RandomUnderSampler(random_state=0, sampling_strategy=strategy)
X_resampled, y_resampled = rus.fit_resample(X_val, y_val)

# ここまでで紹介した全ての評価指標で評価する
evaluate(X_resampled, y_resampled)

In [ ]:
# 評価用のデータのクラスの分布をPositive: Negative = 10:1にする
strategy = calc_custom_storategy(y_val, weights=(10, 1))
rus = RandomUnderSampler(random_state=0, sampling_strategy=strategy)
X_resampled2, y_resampled2 = rus.fit_resample(X_val, y_val)

evaluate(X_resampled2, y_resampled2)

## 3.16.4 ビジネスインパクトによる閾値調整

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB


# 期待値を算出するための関数
def calc_expected_value(conf_matrix):
    # コスト行列
    TP_COST, FP_COST, TN_COST, FN_COST = 9500, -500, 0, 0
    tp, fn, fp, tn = conf_matrix.flatten()

    # 各項目の出現確率を算出
    number_of_sample = np.sum(conf_matrix)
    tp_rate = tp / number_of_sample
    fp_rate = fp / number_of_sample
    tn_rate = tn / number_of_sample
    fn_rate = fn / number_of_sample
    # 利益の期待値を算出
    return (tp_rate * TP_COST) + (fp_rate * FP_COST) + (tn_rate * TN_COST) + (fn_rate * FN_COST)

# 利益曲線を描画する関数
def benefit_curve(clf_list, X_val, y_val):
    # 分類器ごとに利益曲線を描画する
    for clf in clf_list:
        y_val_hat_proba = clf.predict_proba(X_val)

        expected_value_list = []
        thresh_list = []
        for proba in np.sort(y_val_hat_proba[:, 1]):
            # 予測したラベル
            y_val_hat = (y_val_hat_proba[:, 1] > proba).astype(int)
            # 混同行列を作成する
            conf_matrix = confusion_matrix(y_val, y_val_hat, labels=[1, 0])
            # 期待値を算出
            expected_value = calc_expected_value(conf_matrix)
            expected_value_list.append(expected_value)
            thresh_list.append(proba)
        # コストが最大の時のインデックスの番号を取得する
        max_cost_index = expected_value_list.index(max(expected_value_list))
        plt.plot(thresh_list, expected_value_list, label=f"{clf.__class__.__name__}")
        print(f"{clf.__class__.__name__}を使ったときに利益が最大になる閾値: {thresh_list[max_cost_index]}")
    plt.xlabel("threshold")
    plt.ylabel("expected value")
    plt.legend(bbox_to_anchor=(1, 1), loc='upper right', borderaxespad=0, fontsize=10)
    plt.show()

train_df = pd.read_csv('../data/banking_dataset/train.csv', sep=";")

# 前処理する
train_df = label_encoding(train_df)
# 教師ラベルを取り出す
y = train_df.pop('y').values

# 学習用データと検証用データを分離する
X_train, X_val, y_train, y_val = train_test_split(train_df, y, test_size=0.33, random_state=42)
X_train, X_val = standard_scale(X_train, X_val)

# 学習済みの分類器を clf_list に配置する
clf_list = []
for clf in [LogisticRegression(max_iter=200, random_state=2020), GaussianNB()]:
    clf.fit(X_train, y_train)
    clf_list.append(clf)

# 利益曲線を描画する
benefit_curve(clf_list, X_val, y_val)

In [ ]:
# 閾値を指定する
y_val_hat = (clf.predict_proba(X_val)[:, 1] > 0.06102603455402777).astype(int)

# 混同行列を作成する
conf_matrix = confusion_matrix(y_val, y_val_hat, labels=[1, 0])
df_confusion_matrix = pd.DataFrame(conf_matrix, columns=["Positive", "Negative"], index=["Positive", "Negative"])

# 作成した混同行列をヒートマップの形で描画する
sns.heatmap(df_confusion_matrix, fmt="d", annot=True, cmap="Blues", annot_kws={"fontsize": 20})
plt.xlabel("model output class")
plt.ylabel("actual class")
plt.show()